In [ ]:
# for Colab
!pip install botorch

In [7]:
import torch
import botorch
import gpytorch
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement

from torch.distributions import Uniform, Normal, Independent, Distribution
from typing import List

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double # double == float64

# isn't this easier?
# https://github.com/pytorch/botorch/discussions/1444
torch.set_default_dtype(dtype)

# torch.set_default_device(device) # similarly

import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# Question: How to set the lengthscale for each batch of the kernel?
# (will figure this out)

# https://docs.gpytorch.ai/en/stable/kernels.html#gpytorch.kernels.Kernel
# https://docs.gpytorch.ai/en/stable/_modules/gpytorch/kernels/kernel.html#Kernel

# Question: when maximizing the likelihood of batched GP, then does it maximize the
# sum of all the likelihoods of all the batches, or does it maximize them independently?
# I would prefer the latter, but this tutorial
# https://github.com/cornellius-gp/gpytorch/blob/master/examples/08_Advanced_Usage/Simple_Batch_Mode_GP_Regression.ipynb
# maximizes the sum. But I'm not sure what `botorch.fit.fit_gpytorch_mll` does.

In [3]:
def calculate_EI_GP(X_hist, y_hist, X, kernel, fit_params=False):
    """Calculate the exact Expected Improvements at `n_eval` points,
    given `N` histories each of length `n_train`.
    Assumes noise-free observations, so gives a fixed noise level of 1e-6

    Args:
        X_hist: History x values, of shape `(N, n_train, d)`
        y_hist: History y values, of shape `(N, n_train)` or `(N, n_train, 1)`
        X: Evaluation x points, of shape `(N, n_eval, d)`
        kernel: the kernel to use
        fit_params: whether to fit parameters by maximizing the marginal log
            likelihood

    Returns:
        A `(N, n_eval)`-dim tensor of Expected Improvement values at the
        given design points `X`.
    """
    # Get y_hist into (N, n_train, 1) shape so we can give to SingleTaskGP
    if y_hist.dim() == 2:
        y_hist = y_hist.unsqueeze(-1)
    elif y_hist.dim() == 3:
        assert y_hist.size(2) == 1
    else:
        raise AssertionError("y_hist dimension must be 2 or 3")

    assert X_hist.dim() == X.dim() == 3
    assert X_hist.size(0) == y_hist.size(0) == X.size(0) # N=N=N
    assert X_hist.size(1) == y_hist.size(1) # n_train=n_train
    assert X_hist.size(2) == X.size(2) # d=d

    y_hist_var = torch.full_like(y_hist, 1e-6)
    model = SingleTaskGP(X_hist, y_hist, y_hist_var, covar_module=kernel)

    if fit_params:
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)

    # best_f has shape (N,)
    best_f = y_hist.squeeze().amax(1) # unsqueezed so need to squeeze again
    EI_object = ExpectedImprovement(model, best_f=best_f, maximize=True)

    # X currently has shape (N, n_eval, d)
    # Make it have shape (b_1, b_2, 1, d) where (b_1, b_2) = (N, n_eval)
    # The added "1" would be the "q" for "q-batch" in general
    X = X.unsqueeze(2)

    # But also need to swap the batch dimensions to align with what it says here
    # https://botorch.org/docs/batching#batched-models
    X = torch.transpose(X, 0, 1) # Now has shape (n_eval, N, 1, d)

    EI_values = EI_object(X) # shape (n_eval, N)

    EI_values = torch.transpose(EI_values, 0, 1) # need to swap again

    return EI_values # shape (N, n_eval)


In [10]:
gpytorch.models.gp.GP

gpytorch.models.gp.GP

In [227]:
# https://pytorch.org/docs/stable/data.html#map-style-datasets
# https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
# https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset
class GaussainProcessRandomDataset(torch.utils.data.IterableDataset):
    def __init__(self, dimension, n_datapoints:int=None,
                 n_datapoints_random_gen=None,
                 xvalue_distribution: Distribution=None,
                 models: List[botorch.models.model.Model]=None, # gpytorch.models.gp.GP
                 model_probabilities=None, observation_noise:bool=False):
        """Create a dataset that generates random Gaussian Process data.

        Args:
            dimension: The dimension of the feature space d
            n_datapoints: number of (x,y) pairs to generate with each sample;
                could be None
            n_datapoints_random_gen: a callable that returns a random natural
                number that is the number of datapoints.
                Note: exactly one of n_datapoints and n_datapoints_random_gen
                should be speicified (not be None).
            xvalue_distribution: a torch.distributions.Distribution object that
                represents the probability distribution for generating each iid
                value $x \in \mathbb{R}^{dimension}$. Default: iid Uniform(0,1)
            models: a list of models to choose from randomly; defaults to a
                single SingleTaskGP model with the default BoTorch Matern 5/2
                kernel and Gamma priors for the lengthscale, outputscale, and
                noise level.
                It is *assumed* that each model provided is single-output,
                single-batch, and contains *no data*.
            model_probabilities: list of probability of choosing each model
            observation_noise: boolean specifying whether to generate the data
                to include the "observation noise" given by the model's
                likelihood (True), or not (False)
        """
        self.dimension = dimension
        self.observation_noise = observation_noise

        # exacly one of them should be specified; verify this by xor
        assert (n_datapoints is None) ^ (n_datapoints_random_gen is None)
        self.n_datapoints = n_datapoints
        self.n_datapoints_random_gen = n_datapoints_random_gen
        
        if xvalue_distribution is None:
            # m = Normal(torch.zeros(dimension), torch.ones(dimension))
            m = Uniform(torch.zeros(dimension), torch.ones(dimension))
            xvalue_distribution = Independent(m, 1)
        self.xvalue_distribution = xvalue_distribution

        if models is None: # models is None implies model_probabilities is None
            assert model_probabilities is None

            train_X = torch.empty(1, 0, dimension) # (nbatch, n_data, dimension)
            train_Y = torch.empty(1, 0, 1)         # (nbatch, n_data, n_out)
            # Default: Matern 5/2 kernel with gamma priors on
            # lengthscale, outputscale, and noise level
            models = [SingleTaskGP(train_X, train_Y)]
            model_probabilities = torch.tensor([1.0])
        elif model_probabilities is None:
            model_probabilities = torch.full([len(models)], 1/len(models))
        else: # if both were specified, then,
            model_probabilities = torch.as_tensor(model_probabilities)
            assert model_probabilities.dim() == 1
            assert len(models) == len(model_probabilities)
        self.models = models
        self.model_probabilities = model_probabilities
    
    def __iter__(self):
        return self
    
    def __next__(self):
        # pick the model
        # https://discuss.pytorch.org/t/torch-equivalent-of-numpy-random-choice/16146
        model_index = self.model_probabilities.multinomial(num_samples=1, 
                                                           replacement=True)[0]
        model = self.models[model_index]
        random_model = model.pyro_sample_from_prior()

        # pick the number of data points
        if self.n_datapoints is None: # then it's given by a distribution
            n_datapoints = self.n_datapoints_random_gen()
        else:
            n_datapoints = self.n_datapoints

        # generate the x-values
        x_values = self.xvalue_distribution.sample(torch.Size([n_datapoints]))
        assert x_values.dim() == 2 # should have shape (n_datapoints, dimension)
        x_values = x_values.unsqueeze(0) # make have 1 batch for Botorch

        # It is assumed that all the models don't have any data,
        # so the "posterior" is actually just a prior.
        # There are ways to draw from prior if the models don't have data, but
        # they are either hacky or specific to GPyTorch (To be honest, I think
        # both BoTorch and GPyTorch were not designed well)
        # This is a botorch.posteriors.Posterior object.
        prior = random_model.posterior(x_values,
                                observation_noise=self.observation_noise)

        #        n_samples, n_batch, n_datapoints, n_out
        # shape (1,         1,       n_datapoints, 1)
        y_values = prior.sample(torch.Size([1]))
        y_values = y_values.squeeze() # shape (n_datapoints,)

        return x_values, y_values


In [6]:
dimension = 4
m = Uniform(torch.zeros(dimension), torch.ones(dimension))
xvalue_distribution = Independent(m, 1)

ss = xvalue_distribution.sample(torch.Size([20]))
ss.unsqueeze(0)

tensor([[[0.0502, 0.2043, 0.2682, 0.4175],
         [0.2595, 0.4476, 0.9753, 0.9090],
         [0.3216, 0.9895, 0.0275, 0.6854],
         [0.2925, 0.5978, 0.6946, 0.4047],
         [0.0677, 0.4458, 0.4330, 0.7364],
         [0.0247, 0.4647, 0.0088, 0.7085],
         [0.3631, 0.7085, 0.1066, 0.6757],
         [0.9992, 0.8855, 0.3953, 0.6791],
         [0.6873, 0.4191, 0.7254, 0.9278],
         [0.6426, 0.6090, 0.1075, 0.6560],
         [0.1030, 0.1263, 0.2853, 0.5611],
         [0.5030, 0.8255, 0.5013, 0.0890],
         [0.8301, 0.7563, 0.1136, 0.7293],
         [0.5443, 0.8416, 0.1191, 0.1747],
         [0.5138, 0.0511, 0.3795, 0.3961],
         [0.3495, 0.5621, 0.9127, 0.3534],
         [0.4982, 0.4669, 0.2925, 0.1053],
         [0.9870, 0.9217, 0.1165, 0.3036],
         [0.5972, 0.7843, 0.1710, 0.5932],
         [0.6746, 0.5347, 0.1628, 0.3182]]])

In [225]:
kernel = RBFKernel()
x1 = torch.tensor([1.0, 2.0, 3.2])
x2 = torch.tensor([3.0, 4., -1.2])
print(kernel(x1, x2).to_dense())

tensor([[1.5565e-02, 8.5571e-05, 6.4938e-03],
        [3.5321e-01, 1.5565e-02, 2.3545e-05],
        [9.5923e-01, 5.1374e-01, 1.7782e-09]], grad_fn=<RBFCovarianceBackward>)
